* https://plot.ly/python/maps/
* https://data.sfgov.org/Economy-and-Community/Mobile-Food-Facility-Permit/rqzj-sfat

In [9]:
import plotly.plotly as py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [35]:
pd.set_option('display.max_columns', None)

# Importing Data

Get a sense of the data.

In [20]:
sf = pd.read_csv('Mobile_Food_Facility_Permit.csv')
sf.shape

(632, 24)

In [36]:
sf.head()

,locationid,Applicant,FacilityType,cnn,LocationDescription,Address,blocklot,block,lot,permit,Status,FoodItems,X,Y,Latitude,Longitude,Schedule,dayshours,NOISent,Approved,Received,PriorPermit,ExpirationDate,Location
0,1152244,San Francisco's Hometown Creamery,Truck,6109000,GEARY ST: STOCKTON ST to POWELL ST (200 - 299),281 GEARY ST,0314013,0314,013,17MFF-0001,APPROVED,Ice cream,6010423.418,2114694.802,37.787136,-122.407719,http://bsm.sfdpw.org/PermitsTracker/reports/re...,Su:11AM-11PM;Fr/Sa:6PM-11PM,NaN,10/16/2018 12:00:00 AM,2017-01-09,0,07/15/2019 12:00:00 AM,"(37.7871364910861, -122.407718509001)"
1,751253,Pipo's Grill,Truck,5688000,FOLSOM ST: 14TH ST to 15TH ST (1800 - 1899),1800 FOLSOM ST,3549083,3549,083,16MFF-0010,REQUESTED,Tacos: Burritos: Hot Dogs: and Hamburgers,6007856.719,2107724.046,37.767852,-122.416105,http://bsm.sfdpw.org/PermitsTracker/reports/re...,NaN,NaN,NaN,2016-02-04,0,NaN,"(37.7678524427181, -122.416104892532)"
2,735318,Ziaurehman Amini,Push Cart,30727000,MARKET ST: DRUMM ST intersection,5 THE EMBARCADERO,0234017,0234,017,15MFF-0159,REQUESTED,NaN,6013916.720,2117244.027,37.794331,-122.395811,http://bsm.sfdpw.org/PermitsTracker/reports/re...,NaN,NaN,NaN,2015-12-31,0,03/15/2016 12:00:00 AM,"(37.7943310032468, -122.395811053023)"
3,364218,The Chai Cart,Push Cart,9543000,NEW MONTGOMERY ST: AMBROSE BIERCE ST to MISSIO...,79 NEW MONTGOMERY ST,3707014,3707,014,12MFF-0083,SUSPEND,Hot Indian Chai (Tea),6012504.312,2114926.774,37.787890,-122.400535,http://bsm.sfdpw.org/PermitsTracker/reports/re...,Mo-Su:7AM-6PM,NaN,NaN,2012-04-03,0,NaN,"(37.7878896999061, -122.400535326777)"
4,735315,Ziaurehman Amini,Push Cart,4969000,DRUMM ST: MARKET ST to CALIFORNIA ST (1 - 6),1 CALIFORNIA ST,0264004,0264,004,15MFF-0159,REQUESTED,NaN,6013552.570,2116844.500,37.793214,-122.397043,http://bsm.sfdpw.org/PermitsTracker/reports/re...,NaN,NaN,NaN,2015-12-31,0,03/15/2016 12:00:00 AM,"(37.7932137316634, -122.397043036718)"


Each row represents a single applicant, with no duplicates.

In [30]:
sf['Applicant'].count()

632

0    http://bsm.sfdpw.org/PermitsTracker/reports/re...
1    http://bsm.sfdpw.org/PermitsTracker/reports/re...
2    http://bsm.sfdpw.org/PermitsTracker/reports/re...
3    http://bsm.sfdpw.org/PermitsTracker/reports/re...
4    http://bsm.sfdpw.org/PermitsTracker/reports/re...
Name: Schedule, dtype: object